In [0]:
select current_catalog()

In [0]:
use catalog bronze;

In [0]:
create table if not exists managed_default
(width INT, length INT, height INT);

INSERT INTO managed_default
VALUES (3, 2 , 1 )

In [0]:
select * from managed_default

In [0]:
describe extended managed_default


## External Tables

In [0]:
%fs ls 'dbfs:/'

In [0]:
SELECT * FROM csv.`abfss://external@databricksstudysa.dfs.core.windows.net/customers.csv` limit 5;

In [0]:
-- 첫번째 행을 컬럼으로 지정 못함
CREATE TABLE IF NOT EXISTS customers_ext AS 
SELECT * FROM csv.`abfss://external@databricksstudysa.dfs.core.windows.net/customers.csv`;

select * from customers_ext limit 5;

In [0]:
describe extended customers_ext;

In [0]:
CREATE TABLE IF NOT EXISTS customers_ext_with_location
USING CSV
OPTIONS (
  header = "true",
  inferSchema = "true"
)
LOCATION 'abfss://external@databricksstudysa.dfs.core.windows.net/customers.csv';


In [0]:
describe extended customers_ext_with_location;

In [0]:

  SELECT *
  FROM csv.`abfss://external@databricksstudysa.dfs.core.windows.net/customers.csv` limit 5;


In [0]:
CREATE TABLE if not EXISTS customers_managed
USING DELTA
AS
SELECT *
FROM read_files('abfss://external@databricksstudysa.dfs.core.windows.net/customers.csv', format => 'csv', header => true, inferSchema => true);

In [0]:
select * from customers_managed limit 5;

In [0]:
select * from customers_ext_with_location limit 5

In [0]:

select * from delta.`abfss://metastore@databricksstudysa.dfs.core.windows.net/0b444696-1340-4fea-bf07-2eb0eecb0276/tables/89bf4828-e4af-4e84-8d38-5e4bb5ab0504/`


In [0]:
CREATE VOLUME bronze.default.my_volume;

In [0]:
%python
dbutils.fs.ls("/Volumes/bronze/default/my_volume")


In [0]:
%fs ls 'dbfs:/Volumes/bronze/default/my_volume'

In [0]:
SHOW VOLUMES IN bronze.default;

In [0]:
%python
dbutils.fs.mkdirs("/Volumes/bronze/default/my_volume/my_delta_table")

In [0]:
%python
(
  spark.createDataFrame(
    [(1, "Alice", 30), (2, "Bob", 25)],
    ["id", "name", "age"]
  )
  .write
  .format("delta")
  .mode("overwrite")
  .save("/Volumes/bronze/default/my_volume/my_delta_table")
)





In [0]:
%fs ls "/Volumes/bronze/default/my_volume/my_delta_table"


In [0]:
SELECT * 
FROM delta.`/Volumes/bronze/default/my_volume/my_delta_table`;




In [0]:
%fs head 'dbfs:/Volumes/bronze/default/my_volume/my_delta_table/part-00000-61b75178-78a6-4f52-95a2-541c4b2edc60.c000.snappy.parquet'

In [0]:
INSERT INTO external_default
VALUES (3 INT, 2 INT, 1 INT)

In [0]:
%fs ls 'dbfs:/mnt/demo/external_default'

In [0]:
%fs head 'dbfs:/mnt/demo/external_default/part-00000-79aed7f1-7a91-4bd4-bf3f-75355ffcd14d-c000.snappy.parquet'

In [0]:
%fs ls 'dbfs:/mnt/demo/external_default/_delta_log'

In [0]:
%fs head 'dbfs:/mnt/demo/external_default/_delta_log/00000000000000000001.json'

In [0]:
describe extended external_default


## Dropping Tables

In [0]:
drop table managed_default

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/managed_default'

In [0]:
describe history managed_default

In [0]:
DROP TABLE external_default

In [0]:
%fs ls 'dbfs:/mnt/demo/external_default'

In [0]:
describe history external_default

## Creating Schemas

In [0]:
-- create database new_default
create schema new_default;

In [0]:
describe schema extended new_default

In [0]:
describe database extended new_default

In [0]:
USE new_default;

CREATE TABLE managed_new_default
  (width INT, length INT, height INT);
  
INSERT INTO managed_new_default
VALUES (3 INT, 2 INT, 1 INT);

-----------------------------------

CREATE TABLE external_new_default
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_new_default';
  
INSERT INTO external_new_default
VALUES (3 INT, 2 INT, 1 INT);

In [0]:
DESCRIBE EXTENDED managed_new_default
-- dbfs:/user/hive/warehouse/managed_default

In [0]:
DESCRIBE EXTENDED external_new_default

In [0]:
DROP TABLE managed_new_default;
DROP TABLE external_new_default;

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/new_default.db/managed_new_default'

In [0]:
%fs ls 'dbfs:/mnt/demo/external_new_default'

## Creating Schemas in Custom Location

In [0]:
%fs ls 'dbfs:/'

In [0]:
CREATE DATABASE custom
LOCATION 'dbfs:/Shared/schemas/custom.db'

In [0]:
%fs ls 'dbfs:/'

In [0]:
DESCRIBE DATABASE EXTENDED custom

In [0]:
USE custom;

CREATE TABLE managed_custom
  (width INT, length INT, height INT);
  
INSERT INTO managed_custom
VALUES (3 INT, 2 INT, 1 INT);

-----------------------------------

CREATE TABLE external_custom
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_custom';
  
INSERT INTO external_custom
VALUES (3 INT, 2 INT, 1 INT);

In [0]:
DESCRIBE EXTENDED managed_custom

In [0]:
DESCRIBE EXTENDED external_custom

In [0]:
DROP TABLE managed_custom;
DROP TABLE external_custom;

In [0]:
%fs ls 'dbfs:/Shared/schemas/custom.db/managed_custom'

In [0]:
%fs ls 'dbfs:/mnt/demo/external_custom'